## Import the images from github

In [1]:
import requests

In [2]:
#url_image = "https://github.com/MATS64664-2021-Group-2/Sample-Micrographs/tree/main/Example%20Micrographs%20Python%20Course%20Pratheek\chu2.jpg"
url_image = "https://github.com/MATS64664-2021-Group-2/Sample-Micrographs.git"
image_name = "chu2.jpg"

In [3]:
image = requests.get(url_image).content
with open(image_name, 'wb') as handler:
    handler.write(image)